In [ ]:
#pip install -r requirements.txt

In [ ]:
import openai, os, requests
from dotenv import load_dotenv  

# Configure environment variables  
load_dotenv()

In [ ]:
openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version =  os.getenv('AZURE_OPENAI_API_VERSION')

# Azure OpenAI setup
openai.api_base = os.getenv('AZURE_OPENAI_ENDPOINT') # Add your endpoint here
openai.api_key = os.getenv("OPENAI_API_KEY") # Add your OpenAI API key here
deployment_id = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')  # Add your deployment ID here

# Azure AI Search setup


def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.

    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

setup_byod(deployment_id)

In [ ]:



message_text = [{"role": "user", "content": "What are the differences between Lakehouse vs warehouse add headings and subheadings?"}]

completion = openai.ChatCompletion.create(
    messages=message_text,
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
      {
  "type": "AzureCognitiveSearch",
  "parameters": {
                "endpoint": os.getenv("AZURE_SEARCH_ENDPOINT"),
                "key": os.getenv("SEARCH_KEY"),
                "indexName":os.getenv("AZURE_SEARCH_INDEX_NAME"),
                "api-version": os.getenv("AZURE_SEARCH_API_VERSION"),
                "semanticConfiguration": "my-semantic-config",
                "queryType": "vectorSemanticHybrid",
                "fieldsMapping": {
                "contentFieldsSeparator": "\n",
                "contentFields": [
                    "chunk"
                ],
                "filepathField": "name",
                "urlField": "location",
                "vectorFields": [
                    "chunkVector"
                ]
                },
                "inScope": True,
                "roleInformation": "You are an AI assistant that helps people find information.",
                "filter": None,
                "strictness": 3,
                "topNDocuments": 5,
                "embeddingDeploymentName": "text-embedding-ada-002"
  }
}
    ],
    enhancements=None,
    temperature=0,
    top_p=1,
    max_tokens=800,
    stop=None

)
print(completion)